In [1]:
# 모듈 import
from mmcv_custom import runner
from mmcv import Config


from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [2]:
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

# config file 들고오기
#cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
cfg = Config.fromfile('./configs/detectors/detectors_htc_r101_20e_coco.py')

root='../../dataset/'
kfold_root = '../stratified_kfold/'

# dataset config 수정


cfg.data.train.classes = classes
cfg.data.train.img_prefix = root
#cfg.data.train.ann_file = root + 'train.json' # train json 정보
cfg.data.train.ann_file = kfold_root + 'over.json' # train json 정보
#cfg.data.train.pipeline[2]['img_scale'] = (1024,1024) # Resize

#cfg.data.test.classes = classes
#cfg.data.test.img_prefix = root
#cfg.data.test.ann_file = root + 'test.json' # test json 정보
#cfg.data.test.ann_file = kfold_root + 'cv_val_1.json' # test json 정보
#cfg.data.test.pipeline[1]['img_scale'] = (1024,1024) # Resize

cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
#cfg.data.test.ann_file = root + 'test.json' # test json 정보
cfg.data.val.ann_file = kfold_root + 'cv_val_1.json' # test json 정보
cfg.data.val.pipeline[1]['img_scale'] = (1024,1024) # Resize



cfg.data.samples_per_gpu = 2


cfg.seed = 2022
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/htc_detectors_resnext101'

#cfg.model.bbox_head.num_classes = 10
#cfg.model.roi_head.bbox_head[0].num_classes = 10
#cfg.model.roi_head.bbox_head[1].num_classes = 10
#cfg.model.roi_head.bbox_head[2].num_classes = 10


#cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=5, interval=2)


#cfg.runner.max_epochs = 36
#cfg.lr_config = dict(policy='CosineAnnealing', warmup='linear', warmup_iters=1000, warmup_ratio=1.0 / 10, min_lr_ratio=1e-5)

meta = dict()
meta['fp16'] = dict(loss_scale=dict(init_scale=512))
cfg.runner.meta = meta
#cfg.optimizer.lr=0.001


cfg.log_config.hooks = [dict(type='TextLoggerHook'),
                        dict(type='WandbLoggerHook',
                             init_kwargs=dict(project='Object-Detection',
                                              name=f'htc_detectors_resnext101_64_minsu_Ver.0',
                                              entity='bostcamp_ai_tech_level2'))
                       ]



In [3]:
# build_dataset
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!


In [4]:
# dataset 확인
datasets[0]


CocoDataset Train dataset with number of images 3933, and instance counts: 
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| category          | count | category      | count | category        | count | category    | count | category     | count |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
| 0 [General trash] | 3165  | 1 [Paper]     | 4939  | 2 [Paper pack]  | 2130  | 3 [Metal]   | 2274  | 4 [Glass]    | 2295  |
| 5 [Plastic]       | 2372  | 6 [Styrofoam] | 1036  | 7 [Plastic bag] | 4073  | 8 [Battery] | 278   | 9 [Clothing] | 772   |
+-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+

In [ ]:
# 모델 build 및 pretrained network 불러오기
model = build_detector(cfg.model)
model.init_weights()

In [6]:
# 모델 학습
train_detector(model, datasets[0], cfg, distributed=False, validate=True,)

2022-04-06 17:59:31,891 - mmdet - INFO - Start running, host: root@5d948c0036f4, work_dir: /opt/ml/detection/baseline/mmdetection/work_dirs/htc_detectors_resnext101
2022-04-06 17:59:31,892 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) WandbLoggerHook                    
 -------------------- 
before_train_iter:
(VERY_HIGH   ) StepLrUpdaterHook                  
(LOW         ) IterTimerHook          

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


wandb: Currently logged in as: bostcamp_ai_tech_level2 (use `wandb login --relogin` to force relogin)


2022-04-06 18:02:47,561 - mmdet - INFO - Epoch [1][50/1967]	lr: 4.945e-04, eta: 2 days, 3:02:23, time: 3.740, data_time: 0.061, memory: 28526, loss_rpn_cls: 0.2840, loss_rpn_bbox: 0.0678, s0.loss_cls: 0.6983, s0.acc: 89.9609, s0.loss_bbox: 0.1000, s1.loss_cls: 0.2723, s1.acc: 93.7754, s1.loss_bbox: 0.0285, s2.loss_cls: 0.1225, s2.acc: 92.9531, s2.loss_bbox: 0.0056, loss: 1.5790
2022-04-06 18:05:23,658 - mmdet - INFO - Epoch [1][100/1967]	lr: 9.940e-04, eta: 1 day, 22:46:24, time: 3.122, data_time: 0.009, memory: 28526, loss_rpn_cls: 56.4822, loss_rpn_bbox: 131.1554, s0.loss_cls: 2605.7599, s0.acc: 71.3734, s0.loss_bbox: 787.5763, s1.loss_cls: 1072.1007, s1.acc: 74.5691, s1.loss_bbox: 566.1866, s2.loss_cls: 638.8121, s2.acc: 71.6249, s2.loss_bbox: 204.5717, loss: 6062.6451
2022-04-06 18:07:55,444 - mmdet - INFO - Epoch [1][150/1967]	lr: 1.494e-03, eta: 1 day, 20:55:48, time: 3.036, data_time: 0.008, memory: 28612, loss_rpn_cls: 11.5924, loss_rpn_bbox: 14.9673, s0.loss_cls: 570.3837, s0.

RuntimeError: CUDA out of memory. Tried to allocate 260.00 MiB (GPU 0; 31.75 GiB total capacity; 28.17 GiB already allocated; 80.50 MiB free; 28.62 GiB reserved in total by PyTorch)